from Squartini and Arndt:

We further want to quantify whether deviations from zero of the three indices are statistically significant when only a finite amount of sequence data is available to measure the present day nucleotide distribution. To achieve this we compare the distribution of nucleotides, $\rho_\alpha$, of a sequence of length N to the stationary distribution, $\pi_\alpha$, using a $\chi^2$ test with

$$ \chi^2 = N \sum_\alpha  \frac{\rho_\alpha - \pi_\alpha}{\pi_\alpha}$$

This quantity follows a $ \chi^2 $ distribution with 3 degrees of freedom. Deviations from stationarity are significant (with 95% confidence) if $ \chi^2 $ > 7.8147.

In [ ]:
import typing
import numpy

from scipy import stats
from concurrent.futures import ProcessPoolExecutor

from cogent3 import open_data_store

from mdeq.bootstrap import compact_bootstrap_result
from mdeq.stationary_pi import get_stat_pi_via_eigen
from mdeq.utils import load_from_sqldb

import project_paths


RESULT_DIR = project_paths.RESULT_DIR

In [2]:
synthetic_GSN_fits_paths = list((RESULT_DIR / "micro/toe/fg-GSN-toe/").glob("**/*.sqlitedb"))

In [ ]:
def chi_squared(pi, pi_inf, n, motif_order) -> typing.Tuple[float, float]:
    pi_obs = numpy.array([pi[nt] for nt in motif_order])
    pi_exp = numpy.array(pi_inf)

    chi_sum = n * numpy.sum(((pi_obs - pi_exp) ** 2) / pi_exp)
    p = stats.chi2.sf(chi_sum, df=len(motif_order)-1)  

    return chi_sum, p

In [4]:
def squartini_arndt_test(
    hyp_result: compact_bootstrap_result,
) -> typing.Tuple[str, float, float]:
    hyp_result.deserialised_values()
    observed_gn = hyp_result.observed["GN"]
    fg_edge = observed_gn.alignment.info["fg_edge"]
    pi = observed_gn.alignment.probs_per_seq()[fg_edge]
    P = observed_gn.lf.get_psub_for_edge(name=fg_edge)
    pi_inf = get_stat_pi_via_eigen(P)

    chi_2, p = chi_squared(pi = pi, pi_inf=pi_inf, n=len(observed_gn.alignment), motif_order=P.keys())

    return hyp_result.source, chi_2, p

In [5]:
loader = load_from_sqldb()

def process_dm(data_member):
    dm = loader(data_member)
    dm.deserialised_values()
    return squartini_arndt_test(dm)


In [ ]:
for in_path in synthetic_GSN_fits_paths:
    in_dstore = open_data_store(in_path)
    rep = str(in_path).split("/")[-1].split(".")[0]
    out_path = RESULT_DIR / f"micro/squartini-arndt/{rep}.tsv"

    with ProcessPoolExecutor() as ex:
        results = list(ex.map(process_dm, in_dstore.completed))
    
    with open(out_path, "w") as out_file:
        out_file.write('name\tchi2\tchisq_pval\n')
        out_file.writelines(f"{name}\t{chi_2}\t{p}\n" for name, chi_2, p in results)